In [60]:
# If you ae new to go last cell

In [2]:
from catboost import CatBoostClassifier, Pool
import pandas as pd
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [7]:
# Training Data (CC undersampling)===Testing Data (CC undersampling)
train_df = pd.read_csv(".././Training-Testing Dataset/Training Data (RWS Oversampling).csv")
test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data CC undersampling.csv")
# test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data (CC undersampling).csv")

X_train = train_df.drop(columns=["target"])
Y_train = train_df["target"]
X_test = test_df.drop(columns=["target"])
Y_test = test_df["target"]


print(Y_train.value_counts())
print(Y_test.value_counts())


target
Medium_Yes    4492
Low_No        4492
Medium_No     4492
High_No       4492
Low_Yes       4492
High_Yes      4492
Name: count, dtype: int64
target
High_No       509
Low_No        509
Medium_Yes    509
Medium_No     509
High_Yes      304
Low_Yes       188
Name: count, dtype: int64


In [8]:
categorical_features = [col for col in X_train.columns if X_train[col].dtype == 'object' or X_train[col].dtype == 'category']

for col in categorical_features:
    X_train[col] = X_train[col].astype(str)

    X_test[col] = X_test[col].astype(str)

# Y_train["target"] = Y_train["target"].astype(str)
# Y_test["target"] = Y_test["target"].astype(str)

train_pool = Pool(X_train,Y_train,cat_features=categorical_features)
test_pool = Pool(X_test,Y_test,cat_features=categorical_features)

In [52]:
model = CatBoostClassifier(
    iterations=2000,
    depth=8,
    learning_rate=0.05,
    l2_leaf_reg=5,
    loss_function="MultiClass",
    eval_metric="Accuracy",
    early_stopping_rounds=100,
    verbose=100,
    random_state=42
)

model.fit(train_pool,eval_set=test_pool)

y_pred = model.predict(X_test)

print("Accuracy rate ", accuracy_score(Y_test,y_pred)*100)

0:	learn: 0.2793114	test: 0.2274525	best: 0.2274525 (0)	total: 395ms	remaining: 13m 10s
100:	learn: 0.4777011	test: 0.3599684	best: 0.3635285 (68)	total: 31.3s	remaining: 9m 48s
200:	learn: 0.5123553	test: 0.3888449	best: 0.3896361 (198)	total: 1m	remaining: 9m
300:	learn: 0.6227738	test: 0.4924842	best: 0.4924842 (300)	total: 1m 38s	remaining: 9m 15s
400:	learn: 0.6758682	test: 0.5383703	best: 0.5383703 (395)	total: 2m 8s	remaining: 8m 31s
500:	learn: 0.7040665	test: 0.5581487	best: 0.5601266 (485)	total: 2m 37s	remaining: 7m 52s
600:	learn: 0.7294078	test: 0.5771361	best: 0.5783228 (583)	total: 3m 8s	remaining: 7m 18s
700:	learn: 0.7492950	test: 0.5925633	best: 0.5941456 (691)	total: 3m 38s	remaining: 6m 45s
800:	learn: 0.7679207	test: 0.6071994	best: 0.6075949 (789)	total: 4m 9s	remaining: 6m 13s
900:	learn: 0.7824280	test: 0.6166930	best: 0.6178797 (894)	total: 4m 46s	remaining: 5m 50s
1000:	learn: 0.7950059	test: 0.6238133	best: 0.6250000 (983)	total: 5m 22s	remaining: 5m 21s
1100

In [53]:
model.save_model("catboost_marriage_model.cbm")

In [9]:
test_df_cc = pd.read_csv(".././Training-Testing Dataset/Testing data Undersampled(All).csv")

X_test_cc = train_df.drop(columns=["target"])
Y_test_cc = train_df["target"]


for col in categorical_features:
    X_test_cc[col] = X_test_cc[col].astype(str)

test_pool = Pool(X_test_cc,Y_test_cc,cat_features=categorical_features)

In [10]:
y_pred = model.predict(X_test_cc)

print("Accuracy rate ", accuracy_score(Y_test_cc,y_pred)*100)

Accuracy rate  76.34683882457702


In [11]:
test_df_cc = pd.read_csv(".././Training-Testing Dataset/Testing Data CC undersampling.csv")

X_test_cc = train_df.drop(columns=["target"])
Y_test_cc = train_df["target"]


for col in categorical_features:
    X_test_cc[col] = X_test_cc[col].astype(str)

test_pool = Pool(X_test_cc,Y_test_cc,cat_features=categorical_features)

y_pred = model.predict(X_test_cc)

print("Accuracy rate ", accuracy_score(Y_test_cc,y_pred)*100)

Accuracy rate  76.34683882457702


In [15]:
model = CatBoostClassifier()
model = model.load_model("catboost_marriage_model.cbm")
test_df_cc = pd.read_csv(".././Training-Testing Dataset/Testing data Undersampled(All).csv")

X_test_cc = test_df_cc.drop(columns=["target"])
Y_test_cc = test_df_cc["target"]


for col in categorical_features:
    X_test_cc[col] = X_test_cc[col].astype(str)

test_pool = Pool(X_test_cc,Y_test_cc,cat_features=categorical_features)

y_pred = model.predict(X_test_cc)

print("Accuracy rate ", accuracy_score(Y_test_cc,y_pred)*100)

Accuracy rate  73.04964539007092
